# Bike sharing case study
## Problem Statement
Understand the factors affecting the demand for these shared bikes in the American market. The company wants to know:
1. Which variables are significant in predicting the demand for shared bikes.
2. How well those variables describe the bike demands

## Steps 
To understand the problem and propose a solution and sugestions to the bike sharing company post covid restrictions are removed we will be going through following steps on the given data.
1. Reading, Understandting, cleaning and visualizing the data
2. Preparing the data for modelling
    - Train - Test split
    - Rescaling
3. Training the model
    - RSE method
    - Manual elimenation of features
4. Resudaual analysis
5. Prediction and evaluation on the test set
6. Conclusion and suggestions

## Step 1: Reading, Understandting, cleaning and visualizing the data

### Reading

In [1]:
import numpy as np
import pandas as pd

In [3]:
bs = pd.read_csv("day.csv")
bs.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,01-01-2018,1,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,02-01-2018,1,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801
2,3,03-01-2018,1,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,4,04-01-2018,1,0,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,5,05-01-2018,1,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600


### Understanding

- instant: record index
- dteday : date
- season : season (1:spring, 2:summer, 3:fall, 4:winter)
- yr : year (0: 2018, 1:2019)
- mnth : month ( 1 to 12)
- holiday : weather day is a holiday or not (extracted from http://dchr.dc.gov/page/holiday-schedule)
- weekday : day of the week
- workingday : if day is neither weekend nor holiday is 1, otherwise is 0.
+ weathersit : 
    - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
    - 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
    - 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
    - 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
- temp : temperature in Celsius
- atemp: feeling temperature in Celsius
- hum: humidity
- windspeed: wind speed
- casual: count of casual users
- registered: count of registered users
- cnt: count of total rental bikes including both casual and registered

In [4]:
bs.shape

(730, 16)

### Cleaning

### Visualizing

## Step 2: Preparing the data for modelling

### Train - Test split

### Rescalling

## Step 3: Training the model

### RSE methos

### Manual elimenation of features

## Step 4: Resdual analysis

## Step 5: Predection and evaluation on the test set

### Predction

### Evaluation

## Step 6: Observations, Conclusions and Suggestions

### Observations

### Conclusions

### Suggestions